In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score




from sklearn.model_selection import train_test_split
import os
import glob
from tensorflow import keras
from keras.models import Model
from tensorflow.keras.layers import Input
from keras import Sequential
from keras.layers import Dense, Input
from keras.layers import Permute, Reshape, Conv2D, BatchNormalization, MaxPooling2D, Dropout, Activation, Bidirectional, GRU, TimeDistributed, Dense, Conv1D, LSTM,  Flatten
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
import keras_metrics
import keras.backend as K
from datetime import datetime
from sklearn.utils import class_weight
from tensorflow.keras.models import load_model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = np.expand_dims(x_train, axis=-1) / 255.0
x_test = np.expand_dims(x_test, axis=-1) / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


In [4]:

# This model is just for testing. It is just a standard CNN trained on MNIST dataset


# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])




In [5]:
# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=5, validation_split=0.1)


Epoch 1/5
422/422 [==============================] - 24s 55ms/step - loss: 0.2500 - precision: 0.9622 - recall: 0.8930 - val_loss: 0.0780 - val_precision: 0.9824 - val_recall: 0.9762
Epoch 2/5
422/422 [==============================] - 21s 51ms/step - loss: 0.0660 - precision: 0.9830 - recall: 0.9776 - val_loss: 0.0550 - val_precision: 0.9866 - val_recall: 0.9833
Epoch 3/5
422/422 [==============================] - 22s 52ms/step - loss: 0.0474 - precision: 0.9875 - recall: 0.9840 - val_loss: 0.0446 - val_precision: 0.9885 - val_recall: 0.9865
Epoch 4/5
422/422 [==============================] - 22s 52ms/step - loss: 0.0350 - precision: 0.9905 - recall: 0.9883 - val_loss: 0.0469 - val_precision: 0.9873 - val_recall: 0.9853
Epoch 5/5
422/422 [==============================] - 21s 51ms/step - loss: 0.0282 - precision: 0.9919 - recall: 0.9903 - val_loss: 0.0449 - val_precision: 0.9902 - val_recall: 0.9888


In [6]:

# Evaluate the model
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')

print("Precision:", precision)
print("Recall:", recall)

313/313 [==============================] - 2s 6ms/step
Precision: 0.9877879880935623
Recall: 0.9875743955803967


In [7]:
np.dot(y_train, np.array([i for i in range(10)]))

array([5., 0., 4., ..., 5., 6., 8.])

In [8]:
x_train_new, y_train_new = [], []
x_train_base_new, y_train_base_new = [], []
labels = np.dot(y_train, np.array([i for i in range(10)]))

unknown_labels = [7, 8, 9]

for x, y, l in zip(x_train, y_train, labels):
    if l in unknown_labels:
        x_train_new.append(x)
        y_train_new.append(y[7:])
    else:
        x_train_base_new.append(x)
        y_train_base_new.append(y[:7])
x_train_new, y_train_new = np.array(x_train_new), np.array(y_train_new)
x_train_base_new, y_train_base_new = np.array(x_train_base_new), np.array(y_train_base_new)

In [9]:
x_train_new.shape, x_train_base_new.shape, y_train_new.shape, y_train_base_new.shape

((18065, 28, 28, 1), (41935, 28, 28, 1), (18065, 3), (41935, 7))

In [10]:
x_test_new, y_test_new = [], []
x_test_base_new, y_test_base_new = [], []
labels = np.dot(y_test, np.array([i for i in range(10)]))


for x, y, l in zip(x_test, y_test, labels):
    if l in unknown_labels:
        x_test_new.append(x)
        y_test_new.append(y[7:])
    else:
        x_test_base_new.append(x)
        y_test_base_new.append(y[:7])
x_test_new, y_test_new = np.array(x_test_new), np.array(y_test_new)
x_test_base_new, y_test_base_new = np.array(x_test_base_new), np.array(y_test_base_new)

In [11]:
x_test_new.shape, x_test_base_new.shape, y_test_new.shape, y_test_base_new.shape

((3011, 28, 28, 1), (6989, 28, 28, 1), (3011, 3), (6989, 7))

In [12]:

# Build the CNN model
Model = Sequential()
Model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
Model.add(MaxPooling2D((2, 2)))
Model.add(Conv2D(64, (3, 3), activation='relu'))
Model.add(MaxPooling2D((2, 2)))
Model.add(Flatten())
# Model.add(Dense(64, activation='relu'))
Model.add(Dense(10-len(unknown_labels), activation='softmax'))

# Compile the model
Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [13]:
# Train the model
Model.fit(x_train_base_new, y_train_base_new, batch_size=128, epochs=5, validation_split=0.1)


Epoch 1/5
295/295 [==============================] - 15s 49ms/step - loss: 0.2330 - precision_1: 0.9718 - recall_1: 0.8851 - val_loss: 0.0590 - val_precision_1: 0.9854 - val_recall_1: 0.9826
Epoch 2/5
295/295 [==============================] - 14s 48ms/step - loss: 0.0462 - precision_1: 0.9867 - recall_1: 0.9838 - val_loss: 0.0353 - val_precision_1: 0.9931 - val_recall_1: 0.9895
Epoch 3/5
295/295 [==============================] - 14s 49ms/step - loss: 0.0315 - precision_1: 0.9913 - recall_1: 0.9898 - val_loss: 0.0341 - val_precision_1: 0.9926 - val_recall_1: 0.9902
Epoch 4/5
295/295 [==============================] - 14s 48ms/step - loss: 0.0268 - precision_1: 0.9925 - recall_1: 0.9914 - val_loss: 0.0408 - val_precision_1: 0.9916 - val_recall_1: 0.9900
Epoch 5/5
295/295 [==============================] - 15s 49ms/step - loss: 0.0224 - precision_1: 0.9937 - recall_1: 0.9927 - val_loss: 0.0400 - val_precision_1: 0.9919 - val_recall_1: 0.9902


In [14]:
Model.evaluate(x_test_base_new, y_test_base_new, batch_size = 128)

55/55 [==============================] - 1s 14ms/step - loss: 0.0274 - precision_1: 0.9918 - recall_1: 0.9907


[0.02741258777678013, 0.9918349981307983, 0.9906996488571167]

In [15]:
x_test_base_new.shape, y_test_base_new.shape

((6989, 28, 28, 1), (6989, 7))

In [16]:
x_train_new.shape

(18065, 28, 28, 1)

In [17]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 7)                

In [18]:
newModel = Sequential(Model.layers[:-2])

In [19]:
newModel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [20]:
newModel.predict(x_test_base_new).shape

219/219 [==============================] - 1s 5ms/step


(6989, 5, 5, 64)

In [21]:
# Build the CNN model
metaModel = Sequential()
metaModel.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(5, 5, 64)))
metaModel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
metaModel.add(Flatten())
metaModel.add(Dropout(0.5))  # Adding a dropout layer with a rate of 0.5
metaModel.add(Dense(3, activation='softmax'))

# Compile the model
metaModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [22]:
metaModel.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 3, 3, 32)          18464     
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 64)          18496     
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 37,155
Trainable params: 37,155
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 3 way 5 shot setup
def task():
    num_tasks = 20
    tasks = [[] for _ in range(num_tasks)]
    labels = np.dot(y_train_new, np.array([i for i in range(7, 10)]))
    x, y = [[] for _ in unknown_labels], [[] for _ in unknown_labels]
    for a, b in zip(x_test_new, y_test_new):
        x[np.argmax(b)].append(a)
        y[np.argmax(b)].append(b)
        
    X = [np.array(i) for i in x]
    Y = [np.array(i) for i in y]
    del(x, y)
    
    for task in range(num_tasks):
        tasks[task].append(np.concatenate([X[i][task*5:(task+1)*5] for i in range(len(X))]))
        tasks[task].append(np.concatenate([Y[i][task*5:(task+1)*5] for i in range(len(Y))]))
        tasks[task].append(np.concatenate([X[i][num_tasks*5+task*2:num_tasks*5+(task+1)*2] for i in range(len(X))]))
        tasks[task].append(np.concatenate([Y[i][num_tasks*5+task*2:num_tasks*5+(task+1)*2]for i in range(len(Y))]))
    return tasks
    

In [24]:
task_list = task()

In [25]:
metaModel.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 3, 3, 32)          18464     
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 64)          18496     
                                                                 
 flatten_2 (Flatten)         (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 37,155
Trainable params: 37,155
Non-trainable params: 0
_________________________________________________________________


In [26]:
for task in task_list:
    print(task[0].shape, task[1].shape, task[2].shape, task[3].shape)

(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)
(15, 28, 28, 1) (15, 3) (6, 28, 28, 1) (6, 3)


In [27]:
metaModel = Sequential()
metaModel.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(5, 5, 64)))
metaModel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
metaModel.add(Flatten())
metaModel.add(Dropout(0.5))  # Adding a dropout layer with a rate of 0.5
metaModel.add(Dense(3, activation='softmax'))

# Compile the model
metaModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])



In [33]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
precision_metric = keras.metrics.Precision()
recall_metric = keras.metrics.Recall()
optim = tf.keras.optimizers.legacy.Adam(
    learning_rate=0.001
)

for epoch in range(20):
    print(f"start of epoch : {epoch}")
    
    
    with tf.GradientTape(persistent = True) as tape:
        
        for episode in range(5):
            outer_loss = None
            print(f"episode = {episode}")
            for task in task_list[episode*4:(episode+1)*4]:

                clone_newModel = tf.keras.models.clone_model(newModel)
                clone_newModel.set_weights(newModel.get_weights())
                clone_metaModel = tf.keras.models.clone_model(metaModel)
                clone_metaModel.set_weights(metaModel.get_weights())



                y_pred = clone_metaModel(clone_newModel(task[0], training = True), training = True)
                loss = loss_fn(task[1], y_pred)
                gradients = tape.gradient(loss, clone_metaModel.trainable_weights)
                optim.apply_gradients(zip(gradients, clone_metaModel.trainable_weights))



                y_pred_new = clone_metaModel(clone_newModel(task[2], training = True), training = True)
                loss_new = loss_fn(task[3], y_pred_new)


                if outer_loss == None:
                    outer_loss = loss_new
                else:
                    outer_loss += loss_new

            clone_newModel.set_weights(newModel.get_weights())
            clone_metaModel.set_weights(metaModel.get_weights())

            newGradients = tape.gradient(outer_loss, clone_newModel.trainable_weights)
            optim.apply_gradients(zip(newGradients, newModel.trainable_weights))
            newGradients1 = tape.gradient(outer_loss, clone_metaModel.trainable_weights)
            optim.apply_gradients(zip(newGradients1, metaModel.trainable_weights))
        
        
        
        
        
    

start of epoch : 0
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 1
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 2
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 3
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 4
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 5
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 6
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 7
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 8
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 9
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 10
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 11
episode = 0
episode = 1
episode = 2
episode = 3
episode = 4
start of epoch : 12
episode = 0
episode = 1
episod

In [29]:
metaModel.predict(newModel.predict(x_test_base_new)).shape

219/219 [==============================] - 0s 2ms/step


(6989, 3)

In [30]:
print(metaModel(newModel(np.array([task_list[0][0][6]]), training = False), training = False))

tf.Tensor([[1.7247209e-04 2.8704485e-02 9.7112298e-01]], shape=(1, 3), dtype=float32)


In [31]:
print(task_list[0][1][8])

[0. 1. 0.]


In [34]:
metaModel.evaluate(newModel(x_test_new, training = False), y_test_new)

95/95 [==============================] - 0s 2ms/step - loss: 0.5478 - precision_3: 0.8982 - recall_3: 0.8967


[0.5477699637413025, 0.8982036113739014, 0.896712064743042]

In [39]:
new_metric = tf.keras.metrics.CategoricalAccuracy()
y_pred = metaModel(newModel(x_test_new, training = False), training = False)


In [36]:
y_pred.shape

TensorShape([3011, 3])

In [40]:
print(new_metric(y_test_new, y_pred))

tf.Tensor(0.8973763, shape=(), dtype=float32)
